In [1]:
import pandas as pd
import numpy as np
import shutil

In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%run ../utils/sup_functions.ipynb
%run ../utils/excel_tools.ipynb

# Вступление

**Мы свободны в выборе формы нашей отчетности**

Одна из самых распространенных форм - это агрегирующая или сводная таблица. Как правило они предоставляются конечному пользователю в формате excel-файлов.

В python реализваны пакеты, с помощью которых мы можем запустить процесс автоматического формирования таких отчетов.

# Постановка задачи

Допустим у нас есть таблица агрегированных данных по диалогам за опрделенный период. Мы хотим представить эту таблицу нашим клиентам в более удобном формате. А именно:

- Перевести названия колонок на русский язык
- Сделать шапку таблицы многоуровневой, и за счет этого сократить названия некоторых колонок без потери смысла
- Автоматически масштабировать ширину колонок, чтобы клиентам не приходилось подбирать ее самостоятельно
- Добавить группировки на некоторые колонки (они работают как выпадающие списки), чтобы таким образом сократить ее размер без потери содержания
- Добавить фильтр на последнюю строку многоуровневой шапки


- Выделить плохо-оцененные диалоги (rate == 1) красным цветом
- Добавить горизонтальные полосы, разделяющие диалоги за разные даты
- Выделить жирным диалоги оператора, у которого больше всего положительных оценок (5-ок)


# Создание отчета

### Загрузка данных

In [3]:
example = pd.read_csv('../data/input/example.csv')
example.head()

,date,id,participants,theme,rate,oper_name,duration,oper_wait_time,all_mes_count,cust_mes_count,bot_mes_count,oper_mes_count
0,2021-07-01,0,CUST_BOT,THEME2,NaN,NaN,257,0,12,6,6,0
1,2021-07-01,340,CUST_BOT,THEME3,NaN,NaN,763,0,42,21,21,0
2,2021-07-01,339,CUST_BOT_OPER,THEME1,NaN,Полина,928,505,32,12,1,19
3,2021-07-01,338,CUST_BOT,THEME1,NaN,NaN,142,0,22,11,11,0
4,2021-07-01,337,CUST_BOT,THEME2,NaN,NaN,816,0,8,4,4,0


## Оформление шапки

**Какой мы хотим видеть нашу шапку?**

Многоуровневая структура дает нам ряд новых возможностей в плане организации отчета:

1) Мы можем разделить колонки на явные смысловые блоки. И покрасить их в разные цвета

2) Разделение на блоки позволяет давать колонкам короткие названия без потери смысла, ведь он будет исходить из ее расположения (например не писать "Количество сообщений бота", "Количетсов сообщений оператора", а ввести блок "Количество сообщений" и названия колонок "Бота", "Оператора" - как продолжение)

Шапку можно создать вручную и сохранить как excel-файл (data/sup_info/header.xlsx). 
**Но остается вопрос, как правильно прикрепить к ней  данные?**
<img src="../data/sup_info/images/solution.png">

Мы можем считать шапку из файла и даже добавить к ней данные

In [4]:
def replace_unnamed(df, value=''):
    """Функция заменяет значения Unnamed в названиях колонок датафрема df на указанное значение value"""
    unnamed = []
    for row in df.columns:
        for col in row:
            if 'Unnamed:' in col:
                unnamed.append(col)
    df = (df.rename(columns={key: value for key in unnamed}))
    return df

In [5]:
header_path = '../data/sup_info/header.xlsx'
result_path = '../data/results/reports/report.xlsx'

In [6]:
header = pd.read_excel(header_path, header=[0,1])
header = replace_unnamed(header, value='') #
header.head()

Empty DataFrame
Columns: [(Дата, ), (ID диалога, ), (Общая характеристика, Участини диалога), (Общая характеристика, Тема), (Общая характеристика, Оценка), (Общая характеристика, Имя оператора), (Метрики времени , Общая длительность), (Метрики времени , Время реакции оператора), (Количество сообщений, Всего), (Количество сообщений, Клиента), (Количество сообщений, Бота), (Количество сообщений, Оператора)]
Index: []

Однако сохранить готовый отчет в созданом нами формате уже не так легко. Я так и не смог найти простого способа сделать это.

Было бы здорово, если бы мы могли просто **добавить наши данные к шапке снизу и сохранить результат**. Как выяснилось, сделать эту операцию довольно легко

Для этого скопируем файл с шапкой и поместим его в папку reports, при этом изменим название на report.xlsx

In [51]:
shutil.copyfile(header_path, result_path);

### Openpyxl

Пакет для работы с excel-файлами (и не только). С его помощью можно осуществить тонкую настройку представления данных в excel: настроить ширину колонок, добавить на них фильтры и группировки.

In [52]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl import Workbook

In [53]:
sheet_name = 'header'

book = load_workbook(result_path) # добавить ссылку на документацию
sheet = book[sheet_name] # добавить ссылку на документацию

# Здесь важна последовательность - если создать объект writer первым, то он изменит формат файла и объекты book и sheet не смогут создасться
writer = pd.ExcelWriter(result_path, engine='openpyxl') # именно этот объект считывает и сохраняет наши excel-файлы


#### Изменить название листа

In [54]:
print(f'Старое название листа: {book.sheetnames}') # book.sheetnames - получить список листов в файле
sheet.title = 'report'
sheet_name = 'report'
print(f'Новое название листа: {book.sheetnames}')

Старое название листа: ['header']
Новое название листа: ['report']


## Форматирование отчета

Прежде чем сохранить наши данные, вначале отформатируем их нужным нам образом. Этот процесс можно условно разделить на два этапа:

- В первом, мы работаем с объектом sheets и пользуемся инструментарием пакета openpyxl. Здесь мы задаем ширину пколонок, добавляем группировки, фильтры и прочие фишки excel-файлов.
- Во втором, мы работаем с объектом styler из пакета pandas - задаем стилистические параметры. Ширину границ, толщину шрифта, цвет фона и т.д.

### Настройка объекта sheet

#### Масштабируем ширину колонок
Наши требования к ширине колонок исходят из следующих пожеланий:

1) Данные в колонке должны помещаться в ней (ширина колонки в идеале должна быть больше самого длинного элемента в колонке), 

2) Название колонки помещается в ней (в случае с многоуровневой шапки нужно учесть длинну заголовков на каждом уровне, при этом ширина должна быть немного больше названия - возьмем 5 пикселей)

3) Колонки должны помещатся в экране, значит их ширина не должна превышать 80 пикселей

In [55]:
 max([len(el) for el in header_col])

20

In [56]:
width_limit = 80
additional_width = 5 

widths = dict()

for ex_col, header_col in zip(example.columns, header.columns):
    series = example[ex_col]

    col_width = max(
        series.astype(str).map(len).max() + additional_width, # саммый длинный элемент в колонке
        max([len(el) for el in header_col]) + additional_width # саммая длинная запись в названии колонки. 
        # Например в ('Количество сообщений', 'Оператора'), самой длинной выбереться 'Количество сообщений' (20 символов) 
    )

    widths[ex_col] = min([col_width, width_limit])
widths

{'date': 15,
 'id': 15,
 'participants': 25,
 'theme': 25,
 'rate': 25,
 'oper_name': 25,
 'duration': 23,
 'oper_wait_time': 28,
 'all_mes_count': 25,
 'cust_mes_count': 25,
 'bot_mes_count': 25,
 'oper_mes_count': 25}

In [57]:
# Выставляем ширину каждой колонки. Здесь используем функцию get_column_letter, которая принимает на вход номер колонки (начиная с единицы) и возвращает ее буквенное обозначение
for col, col_number in zip(widths, range(1, len(widths)+1)):
    sheet.column_dimensions[get_column_letter(col_number)].width = widths[col]

### Группируем колонки
Мы хотим сгруппировать колонки блока "Количество сообщений", чтобы их можно было свернуть

<img src="../data/sup_info/images/group.png">
<img src="../data/sup_info/images/group_hidden.png">

In [58]:
# Определяем колоки, которые относятся к блоку "Количество сообщений"
group = [
    get_column_letter(col_number) for col, col_number in zip(header.columns, range(1, len(header.columns)+1)) 
    if 'Количество сообщений' in col
]
print(group)


['I', 'J', 'K', 'L']


In [59]:
# Добавляем группировку на эти колонки (передаем первое и последнее значение из списка). Флаг hidden=True сформачивает группу как на картинке выше
sheet.column_dimensions.group(group[1],group[-1], hidden=True)

### Добавляем фильтр

Допустим наш клиент захочит воспользоваться фильтрами excel. В случае многоуровневых шапок возникает вопрос, на какую строку нацепить фильтрацию, чтобы все работало правильно? Ответ - на последнюю, однако до этого еще нужно догадаться. Чтобы избавить клиента от этих затруднений, лучше добавить фильтрацию еще на этапе программирования  

In [60]:
# Определяем номер последней строки в header-е
header_depth = len(header.columns[0])
# Определяем номера колонок, на которые добавить фильтр (в нашем случае это все колонки)
cols_numbers = range(1, len(header.columns)+1)

# Формируем секстовый промежуток, на который добавиться фильтр
filter_span = f"{get_column_letter(cols_numbers[0])}{header_depth}:{get_column_letter(cols_numbers[-1])}{header_depth}"
print(filter_span)

A2:L2


In [61]:
# Добавляем настройку фильтра
sheet.auto_filter.ref = filter_span

### Настраиваем стиль таблицы

Для настройки вида таблицы используется объекта pandas-а **styler**.
Мы можем настраивать стиль каждой строки или столбца в таблице. При этом стиль может включать в себя сразу несколько атрибутов стиля(толщина границ, цвет фона и шрифта, выделение текста).
В таких случаях, удобно создать словарь, где ключом будет идентификатор строки, а значением - набор атрибутов, которые мы будем применять к этой строке.  

In [62]:
# Создаем словарь, где для каждого идентификатора строки (пока что) проставлена пустая строка, тоесть отсутствие каких либо параметров.
style = {ind: '' for ind in example.index}

#### Выделить плохо-оцененные диалоги (rate == 1) красным цветом

In [63]:
# Выделяем интересующие нас идентификаторы строк
bad_rated_rows = example[example['rate'] == 1].index

# Определяем атрибут, который мы будем применять к найденым строкам
attr = 'background-color: red'

# Добавляем к стилю каждого идентификатора атрибут (атрибуты добавляются через точку с запятой)
for ind in bad_rated_rows:
    style[ind] += f'; {attr}'

#### Добавить горизонтальные полосы, разделяющие диалоги за разные даты

In [64]:
last_dates_rows = example[
    example['id'].isin(example.groupby('date', as_index=False).last().id.unique())
].index

attr = 'border-bottom-style: solid; border-bottom-width: 10px'

for ind in last_dates_rows:
    style[ind] += f'; {attr}'

#### Выделить жирным имя оператора, у которого больше всего положительных оценок (5-ок)

In [65]:
best_operator_rows = example[
    example['oper_name'] == example[example['rate'] == 5]['oper_name'].value_counts().index[0]
].index
attr = 'font-weight: bold'

for ind in best_operator_rows:
    style[ind] += f'; {attr}'

#### Для каждой строки нашей таблицы применяем соответствующей ей стиль из словаря

Для большенства строк стиль набор атрибутов остался равен пустой строке и они не будут выделены.

In [66]:
example_styled = example.style.apply(lambda row: [style[row.name]]*len(row), axis=1)

### Сохраняем отформатированный excel-файл

In [67]:
# По неясной мне причине именно такая последовательность комманд правильно применяет формат к файлу
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
writer.sheet = sheet

# Сохраняем отформатированный файл вместе со стилизованной таблицей
example_styled.to_excel(writer,
                 sheet_name=sheet_name, 
                 index=False, 
                 header=False, 
                 startrow=len(header.columns[0]))
writer.close()

### В итоге, мы получаем отчет желаемого вида:
<img src="../data/sup_info/images/final.png">

**Отформатированный полностью в автоматическом режиме**